# Problem setting

In this tutorial, we demonstrate how graph neural networks can be used for recommendation. Here we focus on item-based recommendation model. This method in this tutorial recommends items that are similar to the ones purchased by the user. We demonstrate the recommendation model on the MovieLens dataset.

# Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works with Pytorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL, we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [ ]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Load the rest of necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse as spsp

# Dataset

We use the MoiveLens dataset for demonstration because it is commonly used for recommendation models. In this dataset, there are two types of nodes: users and movies. The movie nodes have three attributes: year, title and genre. There are ratings between user nodes and movie nodes. Each rating has a timestamp. In our recommendation model, we don't consider ratings and timestamps.

**Note**: It is not necessarily the best dataset to demonstrate the power of GNN for recommendation. We have prepared the dataset to simplify the demonstration.

To run the data preprocessing script, a user needs to download the English dictionary of the stanfordnlp package first. However, the following command only needs to run once.

In [ ]:
# Please uncomment the two commands when the tutorial is run for the first time.
#import stanfordnlp
#stanfordnlp.download('en')

Load the MovieLens dataset.

In [ ]:
from movielens import MovieLens
data = MovieLens('.')

Calculate some statistics of the dataset.

In [ ]:
ratings = data.ratings
user_id = np.array(ratings['user_idx'])
movie_id = np.array(ratings['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
num_users, num_movies = user_movie_spm.shape
print('#user-movie iterations:', len(movie_id))
print('#users:', num_users)
print('#movies:', num_movies)

Split the dataset into training, validation and testing sets. In the validation and testing dataset, each user has an item to predict.

In [ ]:
def pick_test(user_movie_spm):
    users = user_movie_spm.row
    movies = user_movie_spm.col
    picks = np.zeros(shape=(len(users)))
    user_movie_spm = user_movie_spm.tocsr()
    indptr = user_movie_spm.indptr
    valid_set = np.zeros(shape=(num_users))
    test_set = np.zeros(shape=(num_users))
    for i in range(user_movie_spm.shape[0]):
        start_idx = indptr[i]
        end_idx = indptr[i+1]
        idx = np.random.choice(np.arange(start_idx, end_idx), 2, replace=False)
        valid_set[i] = movies[idx[0]]
        picks[idx[0]] = 1
        test_set[i] = movies[idx[1]]
        picks[idx[1]] = 1
    users = users[picks == 0]
    movies = movies[picks == 0]
    return spsp.coo_matrix((np.ones((len(users),)), (users, movies))), valid_set, test_set

orig_user_movie_spm = user_movie_spm.tocsr()
user_movie_spm, valid_set, test_set = pick_test(user_movie_spm)
print('#training size:', user_movie_spm.nnz)
users_valid = np.arange(num_users)
movies_valid = valid_set
users_test = np.arange(num_users)
movies_test = test_set
valid_size = len(users_valid)
test_size = len(users_test)
print('valid set:', valid_size)
print('test set:', test_size)

Save the data split in the file.

In [ ]:
coo = user_movie_spm.tocoo()
train_map = {}
valid_map = {}
test_map = {}
#print the training set.
with open("train.txt","w") as file:
    for row, col in zip(coo.row, coo.col):
        train_map[(row, col)] = 1
        file.write(str(row) + ', ' + str(col) + '\n')
    file.close()
with open('valid.txt', 'w') as file:
    for row, col in enumerate(valid_set):
        valid_map[(row, col)] = 1
        file.write(str(row) + ', ' + str(int(col)) + '\n')
    file.close()
with open('test.txt', 'w') as file:
    for row, col in enumerate(test_set):
        test_map[(row, col)] = 1
        file.write(str(row) + ', ' + str(int(col)) + '\n')
    file.close()

Some of the negative samples are actually positive. Here we try to remove all of the postive ones from the negative set.

In [ ]:
def gen_neg_set(user_movie_spm, neg_sample_size):
    num_users = user_movie_spm.shape[0]
    num_movies = user_movie_spm.shape[1]
    neg_mat = np.zeros(shape=(num_users, neg_sample_size))
    for user in range(num_users):
        movie_set = set()
        while len(movie_set) < neg_sample_size:
            movies = np.random.choice(num_movies, neg_sample_size, replace=False)
            for movie in movies:
                if user_movie_spm[user, movie] == 0:
                    movie_set.add(movie)
                if len(movie_set) == neg_sample_size:
                    break
        neg_mat[user] = np.array(list(movie_set))

    for user, movies in enumerate(neg_mat):
        for idx, movie in enumerate(movies):
            assert user_movie_spm[user, movie] == 0
                
    return neg_mat

neg_valid = gen_neg_set(orig_user_movie_spm.tocsr(), 99)
neg_test = gen_neg_set(orig_user_movie_spm.tocsr(), 99)

Save the negative sets.

In [ ]:
with open('neg_valid.txt', 'w') as file:
    for row, cols in enumerate(neg_valid):
        for col in cols:
            assert (row, col) not in train_map
            assert (row, col) not in valid_map
            assert (row, col) not in test_map
            file.write(str(row) + ', ' + str(int(col)) + '\n')
    file.close()

with open('neg_test.txt', 'w') as file:
    for row, cols in enumerate(neg_test):
        for col in cols:
            assert (row, col) not in train_map
            assert (row, col) not in valid_map
            assert (row, col) not in test_map
            file.write(str(row) + ', ' + str(int(col)) + '\n')
    file.close()

This data split is based on timestamp. Let's not use it for now.

In [ ]:
'''
# The training dataset
user_id = np.array(ratings_train['user_idx'])
movie_id = np.array(ratings_train['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
assert num_users == user_movie_spm.shape[0]
assert num_movies == user_movie_spm.shape[1]
train_size = len(user_id)
print('#training size:', train_size)

# The validation and testing dataset
users_valid = ratings[ratings['valid_mask']]['user_idx'].values
movies_valid = ratings[ratings['valid_mask']]['movie_idx'].values
users_test = ratings[ratings['test_mask']]['user_idx'].values
movies_test = ratings[ratings['test_mask']]['movie_idx'].values
valid_size = len(users_valid)
test_size = len(users_test)
print('valid set:', valid_size)
print('test set:', test_size)
'''

# The recommendation model

At large, the model first learns item embeddings from the user-item interaction dataset and use the item embeddings to recommend users similar items they have purchased. To learn item embeddings, we first need to construct an item similarity graph and train GNN on the item graph.

There are many ways of constructing the item similarity graph. Here we use the [SLIM model](https://dl.acm.org/citation.cfm?id=2118303) to learn item similarity and use the learned result to construct the item graph. The resulting graph will have an edge between two items if they are similar and the edge has a weight that represents the similarity score.

After the item similarity graph is constructed, we run a GNN model on it and use the vertex connectivity as the training signal to train the GNN model. The GNN training procedure is very similar to the link prediction task in [the previous section](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

## Construct the movie graph with SLIM
SLIM is an item-based recommendation model. When training SLIM on a user-item dataset, it learns an item similarity graph. This similarity graph is the item graph we construct for the GNN model.

Please follow the instruction on the [SLIM github repo](https://github.com/KarypisLab/SLIM) to install SLIM.

The SLIM only needs to run once and can be saved on the disk for future experiments.

In [ ]:
from SLIM import SLIM, SLIMatrix
model = SLIM()
params = {'algo': 'cd', 'nthreads': 2, 'l1r': 2, 'l2r': 1}
trainmat = SLIMatrix(user_movie_spm.tocsr())
model.train(params, trainmat)
model.save_model(modelfname='slim_model.csr', mapfname='slim_map.csr')

Evaluate the performance of the SLIM model.

In [ ]:
candidates = {}
neg_sample_size = neg_test.shape[1]
for i, neg_items in enumerate(neg_test):
    candidates[i] = np.zeros(shape=(neg_items.shape[0] + 1,), dtype=np.int64)
    candidates[i][:neg_sample_size] = neg_items
    candidates[i][-1] = int(movies_test[i])
rcmd_list = model.predict(trainmat, nrcmds=10, negitems=candidates, nnegs=100)
rcmd_mat = np.ones(shape=(num_users, len(rcmd_list[0])), dtype=np.int64)
for i, rcmd in rcmd_list.items():
    rcmd_mat[i] = rcmd

def test(rcmd_mat, true_list):
    hits = 0
    true_list = np.expand_dims(true_list, 1)
    hits = np.sum(rcmd_mat == true_list)
    #for u, rlist in rcmd_list.items():
    #    hits += true_list[u] in rlist
    print("Number of test(valid) users: %d(%d), hits@%d: %.4f" % 
          (len(true_list), len(rcmd_list), len(rcmd_list[0]), hits / len(rcmd_list)))
    return hits / len(rcmd_list)
test(rcmd_mat, movies_test)

Load the SLIM similarity matrix into DGL. We store the vertex similarity as edge data on DGL.

In [ ]:
from slim_load import read_csr

movie_spm = read_csr('slim_model.csr')
print('#edges:', movie_spm.nnz)
print('most similar:', np.max(movie_spm.data))
print('most unsimilar:', np.min(movie_spm.data))

In [ ]:
g = dgl.DGLGraph(movie_spm, readonly=True)
g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)

Construct the item features.

In [ ]:
year = np.expand_dims(data.movie_data['year'], axis=1)
genre = data.movie_data['genre']
title = data.movie_data['title']
features = torch.tensor(np.concatenate((genre, title), axis=1), dtype=torch.float32)
print('#features:', features.shape[1])
in_feats = features.shape[1]

Save everything in pickle

In [ ]:
import pickle
pickle.dump(user_movie_spm, open('movielens_orig_train.pkl', 'wb'))
pickle.dump(g, open('movielens_graph.pkl', 'wb'))
pickle.dump(features, open('movielens_features.pkl', 'wb'))
pickle.dump((valid_set, test_set), open('movielens_eval.pkl', 'wb'))
pickle.dump((neg_valid, neg_test), open('movielens_neg.pkl', 'wb'))

## Load everything from pickle

In [ ]:
import pickle
user_movie_spm = pickle.load(open('movielens_orig_train.pkl', 'rb'))
g = pickle.load(open("movielens_graph.pkl", "rb"))
features = pickle.load(open('movielens_features.pkl', 'rb'))
valid_set, test_set = pickle.load(open('movielens_eval.pkl', 'rb'))
neg_valid, neg_test = pickle.load(open('movielens_neg.pkl', 'rb'))

in_feats = features.shape[1]
num_users = user_movie_spm.shape[0]
num_movies = user_movie_spm.shape[1]

users_valid = np.arange(num_users)
movies_valid = valid_set
users_test = np.arange(num_users)
movies_test = test_set

## GNN models

We run GNN on the item graph to compute item embeddings. In this tutorial, we use a customized [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) model to compute node embeddings. The original GraphSage performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.

The original GraphSage model treats each neighbor equally. However, the SLIM model learns the item similarity based on the user-item iteration. The GNN model should take the similarity into account. Thus, we customize the GraphSage model in the following fashion. Instead of aggregating all neighbors equally, we aggregate neighbors embeddings rescaled by the similarity on the edges. Thus, the aggregation step is defined as follows:

$$h_{N(v)}^{(l)} \gets \Sigma_{u \in N(v)}({h_u^{(l-1)} * s_{uv}}),$$

where $s_{uv}$ is the similarity score between two vertices $u$ and $v$.

The GNN model has multiple layers. In each layer, a vertex accesses its direct neighbors. When we stack $k$ layers in a model, a node $v$ access neighbors within $k$ hops. The output of the GNN model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

We implement the computation in each layer of the customized GraphSage model in `SAGEConv` and implement the multi-layer model in `GraphSAGEModel`.

In [ ]:
from sageconv import SAGEConv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type,
                                    feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(SAGEConv(n_hidden, out_dim, aggregator_type,
                                    feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h, g.edata['similarity'])
        return h

## Train Item Embeddings

We train the item embeddings with the edges in the item graph as the training signal. This step is very similar to the link prediction task in the [basic applications](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

Because the MovieLens dataset has sparse features (both genre and title are stored as multi-hot encoding). The sparse features have many dimensions. To run GNN on the item features, we first create an encoding layer to project the sparse features to a lower dimension. 

In [ ]:
class EncodeLayer(nn.Module):
    def __init__(self, in_feats, num_hidden):
        super(EncodeLayer, self).__init__()
        self.proj = nn.Linear(in_feats, num_hidden)

    def forward(self, feats):
        return self.proj(feats)

We use the FISM model to train.

In [ ]:
beta = 0
gamma = 0

class FISM(nn.Module):
    def __init__(self, user_movie_spm, gconv_p, gconv_q, in_feats, num_hidden):
        super(FISM, self).__init__()
        self.num_users = user_movie_spm.shape[0]
        self.num_movies = user_movie_spm.shape[1]
        self.b_u = nn.Parameter(torch.zeros(num_users))
        self.b_i = nn.Parameter(torch.zeros(num_movies))
        self.user_deg = torch.tensor(user_movie_spm.dot(np.ones(num_movies)))
        values = user_movie_spm.data
        indices = np.vstack((user_movie_spm.row, user_movie_spm.col))
        indices = torch.LongTensor(indices)
        values = torch.FloatTensor(values)
        self.user_item_spm = torch.sparse_coo_tensor(indices, values, user_movie_spm.shape)
        self.users = user_movie_spm.row
        self.movies = user_movie_spm.col
        self.ratings = user_movie_spm.data
        self.encode_p = EncodeLayer(in_feats, num_hidden)
        self.encode_q = EncodeLayer(in_feats, num_hidden)
        self.gconv_p = gconv_p
        self.gconv_q = gconv_q

    def _est_rating(self, P, Q, user_idx, item_idx):
        bu = self.b_u[user_idx]
        bi = self.b_i[item_idx]
        user_emb = torch.sparse.mm(self.user_item_spm, P)
        user_emb = user_emb[user_idx] / torch.unsqueeze(self.user_deg[user_idx], 1)
        tmp = torch.mul(user_emb, Q[item_idx])
        r_ui = bu + bi + torch.sum(tmp, 1)
        return r_ui
    
    def est_rating(self, g, features, user_idx, item_idx, neg_item_idx):
        P = self.gconv_p(g, self.encode_p(features))
        Q = self.gconv_q(g, self.encode_q(features))
        #P = self.encode_p(features)
        #Q = self.encode_q(features)
        r = self._est_rating(P, Q, user_idx, item_idx)
        neg_sample_size = len(neg_item_idx) / len(user_idx)
        neg_r = self._est_rating(P, Q, np.repeat(user_idx, neg_sample_size), neg_item_idx)
        return torch.unsqueeze(r, 1), neg_r.reshape((-1, int(neg_sample_size)))

    def loss(self, P, Q, r_ui, neg_r_ui):
        diff = 1 - (r_ui - neg_r_ui)
        return torch.sum(torch.mul(diff, diff)/2) \
            + beta/2 * torch.sum(torch.mul(P, P) + torch.mul(Q, Q)) \
            + gamma/2 * (torch.sum(torch.mul(self.b_u, self.b_u)) + torch.sum(torch.mul(self.b_i, self.b_i)))

    def forward(self, g, features, neg_sample_size):
        P = self.gconv_p(g, self.encode_p(features))
        Q = self.gconv_q(g, self.encode_q(features))
        #P = self.encode_p(features)
        #Q = self.encode_q(features)
        tot = len(self.users)
        pos_idx = np.random.choice(tot, int(tot/10))
        user_idx = self.users[pos_idx]
        item_idx = self.movies[pos_idx]
        neg_item_idx = np.random.choice(self.num_movies, len(pos_idx) * neg_sample_size)
        r_ui = self._est_rating(P, Q, user_idx, item_idx)
        neg_r_ui = self._est_rating(P, Q, np.repeat(user_idx, neg_sample_size), neg_item_idx)
        r_ui = torch.unsqueeze(r_ui, 1)
        neg_r_ui = neg_r_ui.reshape((-1, int(neg_sample_size)))
        return self.loss(P, Q, r_ui, neg_r_ui)

We evaluate the performance of the trained item embeddings in the item-based recommendation task. We use the last item that a user purchased to represent the user and compute the similarity between the last item and a list of items (an item the user will purchase and a set of randomly sampled items). We calculate the ranking of the item that will be purchased among the list of items.

In [ ]:
def RecValid(model, g, features):
    model.eval()
    with torch.no_grad():
        neg_movies_eval = neg_valid[users_valid]
        r, neg_r = model.est_rating(g, features, users_valid, movies_valid, neg_movies_eval.flatten())
        hits10 = (torch.sum(neg_r > r, 1) <= 10).numpy()
        print('HITS@10:{:.4f}'.format(np.mean(hits10)))
        return np.mean(hits10)
    
def RecTest(model, g, features):
    model.eval()
    with torch.no_grad():
        neg_movies_eval = neg_test[users_test]
        r, neg_r = model.est_rating(g, features, users_test, movies_test, neg_movies_eval.flatten())
        hits10 = (torch.sum(neg_r > r, 1) <= 10).numpy()
        print('HITS@10:{:.4f}'.format(np.mean(hits10)))
        return np.mean(hits10)

Now we put everything in the training loop.

In [ ]:
#Model hyperparameters
n_hidden = 32
n_layers = 1
dropout = 0.5
aggregator_type = 'sum'

# create GraphSAGE model
gconv_p = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

gconv_q = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

model = FISM(user_movie_spm, gconv_p, gconv_q, in_feats, n_hidden)


# Training hyperparameters
weight_decay = 1e-2
n_epochs = 1000
lr = 1e-3
neg_sample_size = 10

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
prev_acc = 0
for epoch in range(n_epochs):
    model.train()
    loss = model(g, features, neg_sample_size)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch {:05d} | Loss {:.4f}".format(epoch, loss.item()))
    
    acc = RecValid(model, g, features)

print()
# Let's save the trained node embeddings.
RecTest(model, g, features)